In [8]:
# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# keras imports
from keras.applications.densenet import DenseNet169 , preprocess_input


from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import GlobalAveragePooling2D

# other imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from __future__ import print_function

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from lightgbm import LGBMClassifier
import sklearn.metrics as metrics

import numpy as np
from numpy import mean
from numpy import std
import seaborn as sns
import matplotlib.pyplot as plt

import pickle
import glob
import cv2
import h5py
import os
import json
import datetime
import time


# load the user configs , from config.json


In [9]:
# load the user configs

# use your own /config/config.json path
with open('/content/drive/MyDrive/covid19/config/config.json') as f:    
  config = json.load(f)


# config variables
model_name      = config["model"]
weights         = config["weights"]
include_top     = config["include_top"]
train_path      = config["train_path"]
features_path   = config["features_path"]
labels_path     = config["labels_path"]
test_size       = config["test_size"]
model_path      = config["model_path"]


# Use DenseNet169 as Model

In [10]:

# DenseNet169 as model 

if model_name == "DenseNet169":
  base_model = DenseNet169(include_top=include_top, weights=weights, input_tensor=Input(shape=(224,224,3)), input_shape=(224,224,3), pooling="avg")
  model = Model(base_model.input, base_model.get_layer('avg_pool').output)
  image_size = (224, 224)

else:
  base_mode = none


print("[INFO] successfully loaded base model and model...")


[INFO] successfully loaded base model and model...


# Exteract features

In [ ]:


# path to training dataset
train_labels = os.listdir(train_path)

# encode the labels
print("[INFO] encoding labels...")
le = LabelEncoder()
le.fit([tl for tl in train_labels])

# variables to hold features and labels
features = []
labels   = []



# loop over all the labels in the folder
count = 1
for i, label in enumerate(train_labels):
  cur_path = train_path + "/" + label
  count = 1
  for image_path in glob.glob(cur_path + "/*"):
    img = image.load_img(image_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    features.append(flat)
    labels.append(label)
    print("[INFO] processed - " + str(count))
    count += 1
  print("[INFO] completed label - " + label)

# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)

# get the shape of training labels
print("[STATUS] training labels: {}".format(le_labels))
print("[STATUS] training labels shape: {}".format(le_labels.shape))

# save features and labels
h5f_data = h5py.File(features_path, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File(labels_path, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

# save model and weights
model_json = model.to_json()
with open(model_path + str(test_size) + ".json", "w") as json_file:
  json_file.write(model_json)

# save weights
model.save_weights(model_path + str(test_size) + ".h5")
print("[STATUS] saved model and weights to disk..")

print("[STATUS] features and labels saved..")



# Evaluate model

In [ ]:
# load the user configs
# use your own /config/config.json path
with open('/content/drive/MyDrive/covid19/config/config.json') as f:    
  config = json.load(f)

# config variables
test_size        = config["test_size"]
seed             = config["seed"]
features_path    = config["features_path"]
labels_path      = config["labels_path"]
results          = config["results"]
classifier_path  = config["classifier_path"]
train_path       = config["train_path"]
num_classes      = config["num_classes"]
classifier_path  = config["classifier_path"]
selection = config["selection"]
# import features and labels
h5f_data  = h5py.File(features_path, 'r')
h5f_label = h5py.File(labels_path, 'r')

features_string = h5f_data['dataset_1']
labels_string   = h5f_label['dataset_1']

features = np.array(features_string)
labels   = np.array(labels_string)

h5f_data.close()
h5f_label.close()



from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

scoring = {
    'accuracy'    : make_scorer(metrics.accuracy_score),
    'precision'   : make_scorer(metrics.precision_score),
    'sensitivity' : make_scorer(metrics.recall_score),
    'specificity' : make_scorer(metrics.recall_score,pos_label=0)
}


n = int(features.shape[1]*8/100)

rank_1 = 0
fs = SelectKBest(score_func=f_classif, k=n)
X_selected = fs.fit_transform(features, labels)

print("[INFO] features shape: {}".format(X_selected.shape))

# prepare the cross-validation procedure
cv = KFold(n_splits=5, random_state=7803, shuffle=True)

# create model
model = LGBMClassifier(learning_rate=0.2)

# evaluate model
scores_accuracy = cross_val_score(model, X_selected, labels, scoring=scoring['accuracy'], cv=cv, n_jobs=-1)
scores_accuracy  = [ elem*100 for elem in scores_accuracy ]
print(scores_accuracy)
print(' Accuracy: %.2f' % (mean(scores_accuracy)))


print("\n***")
scores_f1_score = cross_val_score(model, X_selected, labels, scoring= 'f1_macro' , cv=cv, n_jobs=-1)
scores_f1_score  = [ elem*100 for elem in scores_f1_score ]
print(scores_f1_score)
print( ' f1_score: %.2f' % (mean(scores_f1_score)))

print("\n***")
scores_precision = cross_val_score(model, X_selected, labels, scoring=scoring['precision'], cv=cv, n_jobs=-1)
scores_precision  = [ elem*100 for elem in scores_precision ]
print(scores_precision)
print( ' precision: %.2f' % (mean(scores_precision)))


print("\n***")
scores_specificity = cross_val_score(model, X_selected, labels, scoring=scoring['specificity'], cv=cv, n_jobs=-1)
scores_specificity  = [ elem*100 for elem in scores_specificity ]
print(scores_specificity)
print( ' specificity: %.2f' % (mean(scores_specificity)))


print("\n***")
scores_sensitivity = cross_val_score(model, X_selected, labels, scoring=scoring['sensitivity'], cv=cv, n_jobs=-1)
scores_sensitivity  = [ elem*100 for elem in scores_sensitivity ]
print(scores_sensitivity)
print( ' sensitivity: %.2f' % (mean(scores_sensitivity)))


y_pred = cross_val_predict(model, X_selected, labels , cv=cv)
cm = confusion_matrix( labels , y_pred)
labels_matrix = sorted(list(os.listdir(train_path)))


print(cm)



fig, ax = plt.subplots()
sns.set(font_scale=2) # Adjust to fit

print("================")


sns.heatmap(cm,
          yticklabels = labels_matrix,
          xticklabels = labels_matrix,
          annot=True,
          cmap='binary',
          fmt=".0f")


ax.tick_params(axis='both', which='major', labelsize=12)  # Adjust to fit


image_format = 'svg' # e.g .png, .svg, etc.
image_name = "/content/drive/MyDrive/covid19/All_data.svg"


plt.title("All Data")
fig.savefig(image_name, format=image_format, dpi=1200)
plt.show()


# confision matrix for each fold

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import plot_confusion_matrix


f = 1
result = {}
labels_matrix = sorted(list(os.listdir(train_path)))
conf_matrix_list_of_arrays = []
acc_a = []

cv = KFold(n_splits=5, random_state=7803, shuffle=True)

for train_ix, test_ix in cv.split(X_selected):
    # get data
    train_X, test_X = X_selected[train_ix], X_selected[test_ix]
    train_y, test_y = labels[train_ix], labels[test_ix]
    # fit model
    model = LGBMClassifier(learning_rate=0.2)
    model.fit(train_X, train_y)
    # evaluate model
    yhat = model.predict(test_X)

    cm = confusion_matrix(test_y, yhat)

    labels_matrix = sorted(list(os.listdir(train_path)))



    fig, ax = plt.subplots()
    sns.set(font_scale=2) # Adjust to fit


    print("================")

    sns.heatmap(cm,
              yticklabels = labels_matrix,
              xticklabels = labels_matrix,
              annot=True,
              cmap='binary',
              fmt=".0f")


    ax.tick_params(axis='both', which='major', labelsize=12)  # Adjust to fit


    image_format = 'svg' # e.g .png, .svg, etc.
    image_name = "/content/drive/MyDrive/covid19/output/confision_matrix_" + str(f) + ".svg"


    plt.title("Fold "+ str(f))
    fig.savefig(image_name, format=image_format, dpi=1200)
    plt.show()
    f +=1
